In [1]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 78.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 68.4 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 

In [2]:
# !pip install --upgrade huggingface-hub -qqq

In [3]:
import os


## 기존 모델 활용

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM # LLM 모델 loading

In [5]:
tokenizer = AutoTokenizer.from_pretrained("beomi/Yi-Ko-6B")
model = AutoModelForCausalLM.from_pretrained("beomi/Yi-Ko-6B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
DDL = 'CREATE TABLE players ( player_id INT PRIMARY KEY AUTO_INCREMENT, username VARCHAR(255) UNIQUE NOT NULL, email VARCHAR(255) UNIQUE NOT NULL, password_hash VARCHAR(255) NOT NULL, date_joined DATETIME NOT NULL, last_login DATETIME );'
Question = '모든 플레이어 정보를 조회해 줘'
SQL = ''
example = f'''
    당신은 sql 생성하는 전문가야.
    DDL 활용해 Question 해결하는 SQL query 생성
    추측 과정은 빼고 SQL만 출력 
    {DDL}
    {Question}
    {SQL}
'''

In [7]:
tokenizer.eos_token_id, tokenizer.pad_token_id

(2, 0)

In [8]:
## 토크나이징
inputs = tokenizer.encode(example, return_tensors='pt')
len(inputs[0])

132

In [9]:
# 모델 예측 수행(생성 글 작성)
outputs = model.generate(inputs
              , max_new_tokens=100
              , pad_token_id = tokenizer.eos_token_id
              , temperature=0.5)
outputs

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


tensor([[    1, 59568,   144,   139, 76851, 71613, 33162, 64069, 76858, 64018,
         65475, 76848,    98,   144,   139, 14499, 59620, 69033, 24983, 65192,
         64018, 16292,  8975, 64069, 76858,   144,   139, 76992, 77248, 64230,
         65028, 67533, 16292, 76822, 64179, 76963, 59568,   144,   139, 57739,
         30589,  3411,   662,  3875, 59593,   619, 11710,  6123,  3811, 15914,
         34341, 58552, 59593,  1625, 26406, 12766,    97, 20576,  1008, 23778,
          1955, 59605,    79,    82,    82, 59604,  8568, 59597, 24849,  5928,
          8987,    97,  3449,  1008, 23778,  1955, 59605,    79,    82,    82,
         59604,  8568, 59597, 24849,  5928,  8987,    97,  8355, 59593, 14529,
          1008, 23778,  1955, 59605,    79,    82,    82, 59604,  5928,  8987,
            97,  3817, 59593,  9522,  1559,   723,  1765,  3013, 13862,  5928,
          8987,    97,  1426, 59593, 17817,   723,  1765,  3013, 13862, 58946,
           144,   139, 71386, 67763, 76816, 68724, 7

In [10]:
# 생성된 텍스트 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
result

" \n    당신은 sql 생성하는 전문가야.\n    DDL 활용해 Question 해결하는 SQL query 생성\n    추측 과정은 빼고 SQL만 출력 \n    CREATE TABLE players ( player_id INT PRIMARY KEY AUTO_INCREMENT, username VARCHAR(255) UNIQUE NOT NULL, email VARCHAR(255) UNIQUE NOT NULL, password_hash VARCHAR(255) NOT NULL, date_joined DATETIME NOT NULL, last_login DATETIME );\n    모든 플레이어 정보를 조회해 줘\n    \n    SELECT * FROM players;\n    모든 플레이어 정보를 조회해 줘\n    \n    SELECT * FROM players WHERE username = 'james';\n    'james'라는 사용자 정보를 조회해 줘\n    \n    SELECT * FROM players WHERE username = 'james' AND password_hash = '12345678';\n    'james'라는 사용자 정보를 조회해 줘 AND '12345678'이라는 패스워드를 가진"

### 모델은 한번에 동작하게 작성

In [11]:
import torch
from transformers import BitsAndBytesConfig, pipeline

def make_inference_pipeline(model_id):
    # 토크나이징
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    # 모델
    model = AutoModelForCausalLM.from_pretrained(model_id
                                                , load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16  # 양자화 정의
                                                , device_map="auto")
    # pipeline : 예측 초기화 설정
    pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)
    return pipe

In [12]:
model_id = "beomi/Yi-Ko-6B"
hf_pipe = make_inference_pipeline(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
results = hf_pipe(example, do_sample=False,
    return_full_text=False, max_length=512, truncation=True)
results

[{'generated_text': "    SELECT * FROM players;\n    \n    SELECT * FROM players WHERE username = 'jonghoon';\n    \n    SELECT * FROM players WHERE email = 'jonghoon@gmail.com';\n    \n    SELECT * FROM players WHERE password_hash = '12345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123"}]

In [16]:
type(hf_pipe), type(pipeline)

(transformers.pipelines.text_generation.TextGenerationPipeline, function)

In [23]:
type(results), type(results[0]), results[0].keys()

(list, dict, dict_keys(['generated_text']))

In [25]:
results[0]['generated_text']

"    SELECT * FROM players;\n    \n    SELECT * FROM players WHERE username = 'jonghoon';\n    \n    SELECT * FROM players WHERE email = 'jonghoon@gmail.com';\n    \n    SELECT * FROM players WHERE password_hash = '12345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123"